In [1]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
import pandas as pd
import numpy as np

C:\Users\User\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


# T1

From : sleepDay_merged.csv, dailyActivity_merged.csv, weightLogInfo_merged.csv, heartrate_seconds_merged.csv (aggregated daily if you can)

In [3]:
sleepDay_merged = pd.read_csv("Kaggle_Dataset/sleepDay_merged.csv") #Contains TotalMinutesAsleep, TotalTimeInBed
dailyActivity_merged = pd.read_csv("Kaggle_Dataset/dailyActivity_merged.csv") #Contains Calories, TotalSteps, VeryActiveMinutes, SedentaryMinutes
weightLogInfo_merged = pd.read_csv("Kaggle_Dataset/weightLogInfo_merged.csv")  # Contains BMI
heartrate_seconds_merged = pd.read_csv("Kaggle_Dataset/heartrate_seconds_merged.csv")

In [4]:
heartrate_seconds_merged['Time'] = pd.to_datetime(heartrate_seconds_merged['Time'])
heartrate_seconds_merged['Date'] = heartrate_seconds_merged['Time'].dt.date
daily_avg_heartrate = heartrate_seconds_merged.groupby(['Id', 'Date'])['Value'].mean().reset_index()
daily_avg_heartrate.rename(columns={'Value': 'AvgHeartRate'}, inplace=True)
daily_avg_heartrate  # Contains AvgHeartRate

,Id,Date,AvgHeartRate
0,2022484408,2016-04-12,75.804177
1,2022484408,2016-04-13,80.337584
2,2022484408,2016-04-14,72.628597
3,2022484408,2016-04-15,80.437382
4,2022484408,2016-04-16,75.960547
...,...,...,...
329,8877689391,2016-05-08,72.550523
330,8877689391,2016-05-09,89.615738
331,8877689391,2016-05-10,71.544377
332,8877689391,2016-05-11,89.149122


To: TotalMinutesAsleep, TotalTimeInBed, Calories, TotalSteps, VeryActiveMinutes, SedentaryMinutes, BMI, AvgHeartRate

In [6]:
sleepDay_merged['SleepDay'] = pd.to_datetime(sleepDay_merged['SleepDay']).dt.date
dailyActivity_merged['ActivityDate'] = pd.to_datetime(dailyActivity_merged['ActivityDate']).dt.date
weightLogInfo_merged['Date'] = pd.to_datetime(weightLogInfo_merged['Date']).dt.date

sleepDay_merged.rename(columns={'SleepDay': 'Date'}, inplace=True)
dailyActivity_merged.rename(columns={'ActivityDate': 'Date'}, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_42648\1983187807.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weightLogInfo_merged['Date'] = pd.to_datetime(weightLogInfo_merged['Date']).dt.date


In [7]:
sleepDay_merged = sleepDay_merged[['Id', 'Date', 'TotalMinutesAsleep', 'TotalTimeInBed']]
dailyActivity_merged = dailyActivity_merged[['Id', 'Date', 'Calories', 'TotalSteps', 'VeryActiveMinutes', 'SedentaryMinutes']]
weightLogInfo_merged = weightLogInfo_merged[['Id', 'Date', 'BMI']]
daily_avg_heartrate = daily_avg_heartrate[['Id', 'Date', 'AvgHeartRate']]

merged_df = pd.merge(sleepDay_merged, dailyActivity_merged, on=['Id', 'Date'], how='outer')
merged_df = pd.merge(merged_df, weightLogInfo_merged, on=['Id', 'Date'], how='outer')
merged_df = pd.merge(merged_df, daily_avg_heartrate, on=['Id', 'Date'], how='outer')

merged_df.dropna(how='all', subset=[
    'TotalMinutesAsleep', 'TotalTimeInBed', 'Calories', 'TotalSteps',
    'VeryActiveMinutes', 'SedentaryMinutes', 'BMI', 'AvgHeartRate'
], inplace=True)

merged_df['SleepEfficiency'] = merged_df['TotalMinutesAsleep'] / merged_df['TotalTimeInBed']
merged_df['SleepEfficiency'] = merged_df['SleepEfficiency'].replace([np.inf, -np.inf], np.nan)
merged_df = merged_df[~((merged_df['SleepEfficiency'] == 0) | (merged_df['SleepEfficiency'].isna()))]

merged_df

,Id,Date,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate,SleepEfficiency
0,1503960366,2016-04-12,327.0,346.0,1985,13162,25,728,NaN,NaN,0.945087
1,1503960366,2016-04-13,384.0,407.0,1797,10735,21,776,NaN,NaN,0.943489
3,1503960366,2016-04-15,412.0,442.0,1745,9762,29,726,NaN,NaN,0.932127
4,1503960366,2016-04-16,340.0,367.0,1863,12669,36,773,NaN,NaN,0.926431
5,1503960366,2016-04-17,700.0,712.0,1728,9705,38,539,NaN,NaN,0.983146
...,...,...,...,...,...,...,...,...,...,...,...
901,8792009665,2016-04-30,343.0,360.0,2896,7174,10,749,NaN,81.464642,0.952778
902,8792009665,2016-05-01,503.0,527.0,1962,1619,0,834,NaN,67.810171,0.954459
903,8792009665,2016-05-02,415.0,423.0,2015,1831,0,916,NaN,67.403919,0.981087
904,8792009665,2016-05-03,516.0,545.0,2297,2421,0,739,NaN,74.371940,0.946789


In [8]:
#forward fill and backward fill BMI 
merged_df.sort_values(by=['Id', 'Date'], inplace=True)
merged_df['BMI_filled'] = merged_df.groupby('Id')['BMI'].ffill()
merged_df['BMI_filled'] = merged_df.groupby('Id')['BMI_filled'].bfill()

null_summary = merged_df.isnull().mean().reset_index()
null_summary.columns = ['Column', 'Percent_Null']
null_summary['Percent_Null'] = null_summary['Percent_Null'] * 100
print("T1")
print(null_summary)
merged_df.describe()

T1
                Column  Percent_Null
0                   Id      0.000000
1                 Date      0.000000
2   TotalMinutesAsleep      0.000000
3       TotalTimeInBed      0.000000
4             Calories      0.000000
5           TotalSteps      0.000000
6    VeryActiveMinutes      0.000000
7     SedentaryMinutes      0.000000
8                  BMI     91.525424
9         AvgHeartRate     55.932203
10     SleepEfficiency      0.000000
11          BMI_filled     77.723971


,Id,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate,SleepEfficiency,BMI_filled
count,4.130000e+02,413.000000,413.000000,413.000000,413.000000,413.000000,413.000000,35.000000,182.000000,413.000000,92.000000
mean,5.000979e+09,419.467312,458.639225,2397.573850,8541.140436,25.188862,712.169492,24.828571,73.615234,0.916769,26.233587
std,2.060360e+09,118.344679,127.101607,762.886326,4156.926990,36.388187,165.957432,4.063695,8.295873,0.087039,5.575708
min,1.503960e+09,58.000000,61.000000,257.000000,17.000000,0.000000,0.000000,22.650000,59.377175,0.498361,22.650000
25%,3.977334e+09,361.000000,403.000000,1850.000000,5206.000000,0.000000,631.000000,23.889999,67.049029,0.912181,22.650000
50%,4.702922e+09,433.000000,463.000000,2220.000000,8925.000000,9.000000,717.000000,24.000000,73.043697,0.943128,24.100000
75%,6.962181e+09,490.000000,526.000000,2926.000000,11393.000000,38.000000,783.000000,24.170000,79.368192,0.960688,28.000000
max,8.792010e+09,796.000000,961.000000,4900.000000,22770.000000,210.000000,1265.000000,47.540001,104.871472,1.000000,47.540001


In [9]:
#reduce df to drop rows with threshold
merged_df = merged_df.dropna(thresh=7)

null_summary = merged_df.isnull().mean().reset_index() 
null_summary.columns = ['Column', 'Percent_Null']
null_summary['Percent_Null'] = null_summary['Percent_Null'] * 100
print("T1")
print(null_summary)

merged_df.describe()

T1
                Column  Percent_Null
0                   Id      0.000000
1                 Date      0.000000
2   TotalMinutesAsleep      0.000000
3       TotalTimeInBed      0.000000
4             Calories      0.000000
5           TotalSteps      0.000000
6    VeryActiveMinutes      0.000000
7     SedentaryMinutes      0.000000
8                  BMI     91.525424
9         AvgHeartRate     55.932203
10     SleepEfficiency      0.000000
11          BMI_filled     77.723971


,Id,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate,SleepEfficiency,BMI_filled
count,4.130000e+02,413.000000,413.000000,413.000000,413.000000,413.000000,413.000000,35.000000,182.000000,413.000000,92.000000
mean,5.000979e+09,419.467312,458.639225,2397.573850,8541.140436,25.188862,712.169492,24.828571,73.615234,0.916769,26.233587
std,2.060360e+09,118.344679,127.101607,762.886326,4156.926990,36.388187,165.957432,4.063695,8.295873,0.087039,5.575708
min,1.503960e+09,58.000000,61.000000,257.000000,17.000000,0.000000,0.000000,22.650000,59.377175,0.498361,22.650000
25%,3.977334e+09,361.000000,403.000000,1850.000000,5206.000000,0.000000,631.000000,23.889999,67.049029,0.912181,22.650000
50%,4.702922e+09,433.000000,463.000000,2220.000000,8925.000000,9.000000,717.000000,24.000000,73.043697,0.943128,24.100000
75%,6.962181e+09,490.000000,526.000000,2926.000000,11393.000000,38.000000,783.000000,24.170000,79.368192,0.960688,28.000000
max,8.792010e+09,796.000000,961.000000,4900.000000,22770.000000,210.000000,1265.000000,47.540001,104.871472,1.000000,47.540001


In [10]:
#bfill & ffill heartrate
merged_df['AvgHeartRate_filled'] = merged_df.groupby('Id')['AvgHeartRate'].ffill().bfill()

null_summary = merged_df.isnull().mean().reset_index() 
null_summary.columns = ['Column', 'Percent_Null']
null_summary['Percent_Null'] = null_summary['Percent_Null'] * 100
print("T1")
print(null_summary)

T1
                 Column  Percent_Null
0                    Id      0.000000
1                  Date      0.000000
2    TotalMinutesAsleep      0.000000
3        TotalTimeInBed      0.000000
4              Calories      0.000000
5            TotalSteps      0.000000
6     VeryActiveMinutes      0.000000
7      SedentaryMinutes      0.000000
8                   BMI     91.525424
9          AvgHeartRate     55.932203
10      SleepEfficiency      0.000000
11           BMI_filled     77.723971
12  AvgHeartRate_filled      0.000000


In [11]:
merged_df.drop('AvgHeartRate', axis=1, inplace=True)
merged_df.drop('BMI', axis=1, inplace=True)
merged_df.rename(columns={'BMI_filled': 'BMI', 'AvgHeartRate_filled': 'AvgHeartRate'}, inplace=True)
merged_df

,Id,Date,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,SleepEfficiency,BMI,AvgHeartRate
0,1503960366,2016-04-12,327.0,346.0,1985,13162,25,728,0.945087,22.65,68.656250
1,1503960366,2016-04-13,384.0,407.0,1797,10735,21,776,0.943489,22.65,68.656250
3,1503960366,2016-04-15,412.0,442.0,1745,9762,29,726,0.932127,22.65,68.656250
4,1503960366,2016-04-16,340.0,367.0,1863,12669,36,773,0.926431,22.65,68.656250
5,1503960366,2016-04-17,700.0,712.0,1728,9705,38,539,0.983146,22.65,68.656250
...,...,...,...,...,...,...,...,...,...,...,...
901,8792009665,2016-04-30,343.0,360.0,2896,7174,10,749,0.952778,NaN,81.464642
902,8792009665,2016-05-01,503.0,527.0,1962,1619,0,834,0.954459,NaN,67.810171
903,8792009665,2016-05-02,415.0,423.0,2015,1831,0,916,0.981087,NaN,67.403919
904,8792009665,2016-05-03,516.0,545.0,2297,2421,0,739,0.946789,NaN,74.371940


GAN Synthesise

In [13]:
def synthesiser(data, id_col, date_col, participant_col, start_participant_num, num_new_participants):
    """
    Generate synthetic data for new participants using CTGAN.

    Parameters:
    - data (pd.DataFrame): Original dataset.
    - id_col (str): Column name to be treated as unique row ID (e.g., 'row_id').
    - date_col (str): Column name representing date for each ID.
    - participant_col (str): Column name representing participant IDs.
    - start_participant_num (int): Number to start naming new participants from (e.g., 17 for 'p17').
    - num_new_participants (int): How many new participants to synthesize.

    Returns:
    - pd.DataFrame: Synthetic data for the new participants.
    """
    
    # Step 0: Assign unique row ID if not already
    if id_col not in data.columns:
        data[id_col] = range(len(data))
    
    # Step 1: Setup metadata
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data)
    metadata.update_column(column_name=id_col, sdtype='id')
    metadata.set_primary_key(column_name=id_col)
    metadata.update_column(column_name=participant_col, sdtype='categorical')
    metadata.update_column(column_name=date_col, sdtype='datetime')

    # Step 2: Fit the synthesizer
    synthesizer = CTGANSynthesizer(metadata)
    synthesizer.fit(data)

    # Step 3: Estimate rows per participant and sample synthetic data
    original_participant_count = data[participant_col].nunique()
    rows_per_participant = len(data) // original_participant_count
    synthetic_sample_size = rows_per_participant * num_new_participants
    synthetic_data = synthesizer.sample(synthetic_sample_size)

    # Step 4: Generate new participant IDs
    new_ids = [f"p{i}" for i in range(start_participant_num, start_participant_num + num_new_participants)]

    # Step 5: Build new synthetic dataset
    new_participant_data = pd.DataFrame()

    for new_id in new_ids:
        # Sample synthetic data for each new participant
        subset = synthetic_data.sample(rows_per_participant, replace=True).copy()
        subset[participant_col] = new_id
        # Assign new dates for each sampled row
        date_range = pd.date_range(start="2023-01-01", periods=rows_per_participant, freq='D')
        subset[date_col] = date_range
        new_participant_data = pd.concat([new_participant_data, subset], ignore_index=True)

    # Step 6: Reassign row IDs
    new_participant_data[id_col] = range(1, len(new_participant_data) + 1)

    return new_participant_data


start_participant_num = merged_df['Id'].nunique()
synthetic_data = synthesiser(merged_df, id_col='row_id', date_col='Date', participant_col='Id', start_participant_num=start_participant_num, num_new_participants=100)

print(synthetic_data.head())

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'row_id'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:1250: UserWarning: No 'datetime_format' is present in the metadata for the following columns:
Column Name   sdtype datetime_format
       Date datetime            None
Without this specification, SDV may not be able to accurately parse the data. We recommend adding datetime formats using 'update_column'.
  warnings.

    Id       Date  TotalMinutesAsleep  TotalTimeInBed  Calories  TotalSteps  \
0  p24 2023-01-01               555.0           542.0       843        3741   
1  p24 2023-01-02               539.0           462.0      2686       16008   
2  p24 2023-01-03               601.0           366.0      2949        8682   
3  p24 2023-01-04               680.0            61.0      3130        3257   
4  p24 2023-01-05               303.0           450.0      2244        8081   

   VeryActiveMinutes  SedentaryMinutes  SleepEfficiency  BMI  AvgHeartRate  \
0                 19               527         0.997786  NaN     64.934642   
1                 11               881         0.996494  NaN     68.191648   
2                 68               367         0.976156  NaN     69.102779   
3                  0               603         0.975133  NaN    100.826923   
4                  0               799         0.955712  NaN     73.780569   

   row_id  
0       1  
1       2  
2       3  
3       

In [14]:
final = pd.concat([merged_df, synthetic_data], ignore_index=True)
final.drop('Id', axis=1, inplace=True)
final.drop('Date', axis=1, inplace=True)
final.drop('row_id', axis=1, inplace=True)
final.to_csv("Final_Dataset/sleepDay_merged.csv (derived).csv", index=False)
final.describe()

,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,SleepEfficiency,BMI,AvgHeartRate
count,2113.000000,2113.000000,2113.000000,2113.000000,2113.000000,2113.000000,2113.000000,443.000000,2113.000000
mean,531.713204,396.176053,2190.917653,9735.676290,31.368670,594.177000,0.919526,25.595800,73.269036
std,121.262379,144.926582,832.557952,4808.234988,43.870169,215.615082,0.099373,3.100260,7.809780
min,58.000000,61.000000,257.000000,17.000000,0.000000,0.000000,0.498361,22.650000,59.377175
25%,455.000000,309.000000,1549.000000,6155.000000,0.000000,481.000000,0.906706,23.889999,67.979681
50%,547.000000,394.000000,2095.000000,9482.000000,12.000000,598.000000,0.949533,25.861630,71.085359
75%,625.000000,490.000000,2729.000000,13231.000000,48.000000,728.000000,0.979047,26.175749,78.966372
max,796.000000,961.000000,4900.000000,22770.000000,210.000000,1265.000000,1.000000,47.540001,104.871472


In [15]:
final

,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,SleepEfficiency,BMI,AvgHeartRate
0,327.0,346.0,1985,13162,25,728,0.945087,22.650000,68.656250
1,384.0,407.0,1797,10735,21,776,0.943489,22.650000,68.656250
2,412.0,442.0,1745,9762,29,726,0.932127,22.650000,68.656250
3,340.0,367.0,1863,12669,36,773,0.926431,22.650000,68.656250
4,700.0,712.0,1728,9705,38,539,0.983146,22.650000,68.656250
...,...,...,...,...,...,...,...,...,...
2108,591.0,338.0,1170,13293,36,380,0.947698,26.032929,104.871472
2109,638.0,326.0,1024,16472,15,259,0.970758,NaN,69.770776
2110,524.0,544.0,3101,3708,0,644,0.973533,NaN,71.311658
2111,572.0,309.0,1822,7666,0,722,0.518795,NaN,68.802741
